In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("LEGO brick images v1"))

# Any results you write to the current directory are saved as output.

['11214 Bush 3M friction with Cross axle', '18651 Cross Axle 2M with Snap friction', '2357 Brick corner 1x2x2', '3003 Brick 2x2', '3004 Brick 1x2', '3005 Brick 1x1', '3022 Plate 2x2', '3023 Plate 1x2', '3024 Plate 1x1', '3040 Roof Tile 1x2x45deg', '3069 Flat Tile 1x2', '32123 half Bush', '3673 Peg 2M', '3713 Bush for Cross Axle', '3794 Plate 1X2 with 1 Knob', '6632 Technic Lever 3M']


# Acknowledgements

This kernel draws inspiration from Kaggler **Paul Mooney** image kernels found [here](https://www.kaggle.com/paultimothymooney/kernels)

# Step 1 : Import Modules

In [3]:
import pandas as pd
import numpy as np
from os import listdir
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import seaborn as sns
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

C:\Users\seppe\AppData\Local\Temp\ipykernel_14320\3201384208.py:11: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  from scipy.misc import imresize, imread


ImportError: cannot import name 'imresize' from 'scipy.misc' (c:\Users\seppe\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\misc\__init__.py)

# Step Two : Explore Data

In [ ]:
listdir('../input/lego brick images')[:10]

In [ ]:
listdir('../input/lego brick images/LEGO brick images')[:10]

In [ ]:
listdir('../input/lego brick images/LEGO brick images/train')

In [ ]:
len(listdir('../input/lego brick images/LEGO brick images/train'))

In [ ]:
len(listdir('../input/lego brick images/LEGO brick images/valid'))

In [ ]:
listdir('../input/lego brick images/LEGO brick images/train/3023 Plate 1x2')

# Step 3 :Plot Data

In [ ]:
train_path = '../input/lego brick images/LEGO brick images/train/'

In [ ]:
valid_path ='../input/lego brick images/LEGO brick images/valid/'

In [ ]:
dir_name = '3023 Plate 1x2'

In [ ]:
dir_path = train_path + dir_name + '/'

In [ ]:
image_path = dir_path + '0165.png'

In [ ]:
image_path

In [ ]:
image = cv2.imread(image_path)
plt.figure(figsize=(16,16))
plt.imshow(image)

In [ ]:
image.shape

## Individual Image 2

In [ ]:
image_path = dir_path + '0032.png'
image = cv2.imread(image_path)
plt.figure(figsize=(16,16))
plt.imshow(image)

In [ ]:
image.shape


## Plot Bunch of Images

In [ ]:
path_name = train_path + '/**/*.png'

In [ ]:
imagePatches = glob(path_name, recursive=True)
for filename in imagePatches[0:10]:
    print(filename)

In [ ]:
# Plot Multiple Images
bunchOfImages = imagePatches
i_ = 0
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in bunchOfImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (50, 50)) 
    plt.subplot(5, 5, i_+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1

# Load Images

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
def loadBatchImages(path):
    catList = listdir(path)
    loadedImagesTrain = []
    loadedLabelsTrain = []
    for cat in catList:
        deepPath = path+cat+"/"
        imageList = listdir(deepPath)
        indx = 0
        for images in imageList:                
            img = load_img(deepPath + images)
            img = img_to_array(img)
            loadedLabelsTrain.append(cat)
            loadedImagesTrain.append(img)
    return loadedImagesTrain, loadedLabelsTrain

In [ ]:
loadedImagesTrain, loadedLabelsTrain = loadBatchImages(train_path)

In [ ]:
len(loadedImagesTrain)

In [ ]:
len(loadedLabelsTrain)

In [ ]:
len(np.unique(loadedLabelsTrain))

In [ ]:
#Encode labels with value between 0 and n_classes-1.
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
encoder.fit(loadedLabelsTrain)

In [ ]:
encoded_loadedLabelsTrain = encoder.transform(loadedLabelsTrain)

In [ ]:
len(encoded_loadedLabelsTrain)

In [ ]:
encoded_loadedLabelsTrain[:40]

In [ ]:
encoded_loadedLabelsTrain[1000:1040]

In [ ]:
counts = np.bincount(encoded_loadedLabelsTrain)

In [ ]:
counts

In [ ]:
loadedImagesTrain = np.asarray(loadedImagesTrain)

In [ ]:
type(loadedImagesTrain)

In [ ]:
loadedImagesTrain.shape

In [ ]:
type(encoded_loadedLabelsTrain)

In [ ]:
encoded_loadedLabelsTrain.shape

In [ ]:
loadedImagesValid, loadedLabelsValid = loadBatchImages(valid_path)
loadedImagesValid = np.asarray(loadedImagesValid)
encoder.fit(loadedLabelsValid)
encoded_loadedLabelsValid = encoder.transform(loadedLabelsValid)

# Train and Test Set

In [ ]:
X_train, X_test, Y_train, Y_test = loadedImagesTrain,loadedImagesValid,encoded_loadedLabelsTrain,encoded_loadedLabelsValid

In [ ]:
del loadedImagesTrain,encoded_loadedLabelsTrain
del loadedImagesValid,encoded_loadedLabelsValid

In [ ]:
import gc 

In [ ]:
gc.collect()

## One Hot Encoding

In [ ]:
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(Y_train, num_classes = 16)
y_testHot = to_categorical(Y_test, num_classes = 16)

In [ ]:
X_train.shape

In [ ]:
y_trainHot.shape

# Helper Functions

In [ ]:
# Helper Functions  Learning Curves and Confusion Matrix

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    #plt.clf()
    # summarize history for loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

# Build the Convolutional Network

In [ ]:
batch_size = 128
num_classes = 16
epochs = 10
img_rows,img_cols=200,200
input_shape = (img_rows, img_cols, 3)
e = 2

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,strides=e))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

In [ ]:
a = X_train
b = y_trainHot
c = X_test
d = y_testHot
epochs = 30

In [ ]:
history = model.fit_generator(datagen.flow(a,b, batch_size=32),
                        steps_per_epoch=len(a) / 32, 
                              epochs=epochs,validation_data = [c, d],
                              callbacks = [MetricsCheckpoint('logs')])

In [ ]:
plotKerasLearningCurve()
plt.show()  

In [ ]:
plot_learning_curve(history)
plt.show()

# Vgg16

In [ ]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model, Sequential, model_from_json

In [ ]:
imageSize =200

In [ ]:
pretrained_model_1 = VGG16(include_top=False, input_shape=(imageSize, imageSize, 3))

In [ ]:
base_model = pretrained_model_1 # Topless
optimizer1 = keras.optimizers.Adam()
# Add top layer
x = base_model.output
x = Conv2D(200, kernel_size = (3,3), padding = 'valid')(x)
x = Flatten()(x)
x = Dropout(0.75)(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
# Train top layer
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer1, 
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train,y_trainHot, 
                        epochs=10, 
                        batch_size = 32,
                        validation_data=(X_test,y_testHot), 
                        verbose=1)